# TSA Chapter 1: Time Series Basics and Motivation

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/QuantLet/TSA/blob/main/TSA_Ch1/TSA_ch1_ts_basics/TSA_ch1_ts_basics.ipynb)

This notebook demonstrates:
- Time series motivation: real-world examples from different domains
- S&P 500 comprehensive analysis: prices, returns, ACF, volatility clustering
- Returns distribution: fat tails, Student-t fit, QQ-plot
- Volatility clustering in financial returns

In [ ]:
!pip install yfinance matplotlib numpy scipy statsmodels pandas -q

In [ ]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy import stats
from statsmodels.tsa.stattools import acf

try:
    import yfinance as yf
    YF_AVAILABLE = True
except ImportError:
    YF_AVAILABLE = False

In [ ]:
# Style configuration
COLORS = {
    'blue': '#1A3A6E',
    'red': '#DC3545',
    'green': '#2E7D32',
    'orange': '#E67E22',
    'gray': '#666666',
    'purple': '#8E44AD',
}

plt.rcParams.update({
    'axes.facecolor': 'none',
    'figure.facecolor': 'none',
    'savefig.transparent': True,
    'axes.spines.top': False,
    'axes.spines.right': False,
    'axes.grid': False,
    'font.size': 9,
    'axes.titlesize': 10,
    'axes.labelsize': 9,
    'xtick.labelsize': 8,
    'ytick.labelsize': 8,
    'legend.fontsize': 8,
    'figure.dpi': 150,
    'lines.linewidth': 1.2,
    'axes.edgecolor': '#333333',
    'axes.linewidth': 0.8,
})

np.random.seed(42)

def save_chart(fig, name):
    fig.savefig(f'{name}.pdf', bbox_inches='tight', transparent=True, dpi=150)
    fig.savefig(f'{name}.png', bbox_inches='tight', transparent=True, dpi=150)
    try:
        charts_path = os.path.join('..', '..', '..', 'charts', name)
        fig.savefig(f'{charts_path}.pdf', bbox_inches='tight', transparent=True, dpi=150)
        fig.savefig(f'{charts_path}.png', bbox_inches='tight', transparent=True, dpi=150)
    except Exception:
        pass
    print(f'Saved: {name}.pdf + .png')

def add_legend_below(ax, ncol=3):
    ax.legend(loc='upper center', bbox_to_anchor=(0.5, -0.18), ncol=ncol, frameon=False)

In [ ]:
# Chart: ch1_motivation_real
# Real-world time series examples
np.random.seed(42)
n = 200

fig, axes = plt.subplots(2, 2, figsize=(8, 4.0))

t = np.arange(n)
gdp = 100 + 0.3 * t + 5 * np.sin(2*np.pi*t/40) + np.random.normal(0, 2, n)
axes[0, 0].plot(gdp, color=COLORS['blue'], linewidth=0.8)
axes[0, 0].set_title('GDP (trend + cycle)', fontsize=9, fontweight='bold')
axes[0, 0].set_ylabel('Value')

temp = 15 + 10 * np.sin(2*np.pi*t/12) + np.random.normal(0, 2, n)
axes[0, 1].plot(temp, color=COLORS['red'], linewidth=0.8)
axes[0, 1].set_title('Temperature (seasonal)', fontsize=9, fontweight='bold')

stock = 100 * np.exp(np.cumsum(np.random.normal(0.001, 0.02, n)))
axes[1, 0].plot(stock, color=COLORS['green'], linewidth=0.8)
axes[1, 0].set_title('Stock price (random walk)', fontsize=9, fontweight='bold')
axes[1, 0].set_xlabel('Time')
axes[1, 0].set_ylabel('Value')

elec = 500 + 100 * np.sin(2*np.pi*t/12) + 30 * np.sin(2*np.pi*t/52) + 0.5*t + np.random.normal(0, 20, n)
axes[1, 1].plot(elec, color=COLORS['orange'], linewidth=0.8)
axes[1, 1].set_title('Energy consumption (multiple seasonality)', fontsize=9, fontweight='bold')
axes[1, 1].set_xlabel('Time')

fig.tight_layout(rect=[0, 0.05, 1, 1])
save_chart(fig, 'ch1_motivation_real')
plt.show()

In [ ]:
# Chart: ch1_motivation_everywhere
# Time series in different domains
np.random.seed(42)
n = 150

fig, axes = plt.subplots(2, 3, figsize=(9, 3.5))

labels = ['Financial markets', 'Macroeconomics', 'Energy',
          'Climate/Weather', 'Health', 'Retail/Sales']
colors_list = [COLORS['blue'], COLORS['red'], COLORS['green'],
               COLORS['orange'], COLORS['purple'], COLORS['gray']]

for i, (ax, label, col) in enumerate(zip(axes.flat, labels, colors_list)):
    t = np.arange(n)
    if i == 0:
        y = 100 * np.exp(np.cumsum(np.random.normal(0.001, 0.02, n)))
    elif i == 1:
        y = 100 + 0.3*t + np.random.normal(0, 3, n)
    elif i == 2:
        y = 500 + 100*np.sin(2*np.pi*t/12) + 0.5*t + np.random.normal(0, 20, n)
    elif i == 3:
        y = 15 + 12*np.sin(2*np.pi*t/12) + np.random.normal(0, 2, n)
    elif i == 4:
        y = np.maximum(5 + 0.1*t + 15*np.exp(-((t-75)/10)**2) + np.random.normal(0, 2, n), 0)
    else:
        y = 200 + 50*np.sin(2*np.pi*t/12) + 0.3*t + np.random.normal(0, 15, n)
    ax.plot(y, color=col, linewidth=0.7)
    ax.set_title(label, fontsize=8, fontweight='bold')
    if i >= 3:
        ax.set_xlabel('Time')

fig.tight_layout(rect=[0, 0.05, 1, 1])
save_chart(fig, 'ch1_motivation_everywhere')
plt.show()

In [ ]:
# Chart: ch1_motivation_forecast
# Forecasting illustration
np.random.seed(42)
n = 100
h = 20
y = 50 + 0.3*np.arange(n) + 5*np.sin(2*np.pi*np.arange(n)/12) + np.random.normal(0, 2, n)

t_future = np.arange(n, n+h)
forecast = 50 + 0.3*t_future + 5*np.sin(2*np.pi*t_future/12)
ci_lower = forecast - 2 * np.linspace(2, 5, h)
ci_upper = forecast + 2 * np.linspace(2, 5, h)

fig, ax = plt.subplots(figsize=(7, 3.0))
ax.plot(range(n), y, color=COLORS['blue'], linewidth=0.8, label='Observed data')
ax.plot(range(n, n+h), forecast, color=COLORS['red'], linewidth=1.2, linestyle='--', label='Forecast')
ax.fill_between(range(n, n+h), ci_lower, ci_upper, alpha=0.15, color=COLORS['red'], label='95% CI')
ax.axvline(n, color=COLORS['gray'], linewidth=0.8, linestyle=':', alpha=0.5)
ax.set_title('Time series forecasting', fontsize=10, fontweight='bold')
ax.set_xlabel('Time')
ax.set_ylabel('Value')
add_legend_below(ax, ncol=3)
fig.tight_layout(rect=[0, 0.05, 1, 1])
save_chart(fig, 'ch1_motivation_forecast')
plt.show()

In [ ]:
# Load S&P 500 data for comprehensive analysis
SP500_LOADED = False
if YF_AVAILABLE:
    try:
        df = yf.download('^GSPC', start='2020-01-01', end='2025-12-31', progress=False)
        close = df['Close'].squeeze().dropna()
        prices = close.values
        log_returns = np.diff(np.log(prices))
        dates = close.index
        SP500_LOADED = True
        print(f'S&P 500 loaded: {len(close)} observations')
    except Exception as e:
        print(f'Yahoo Finance failed: {e}')

if not SP500_LOADED:
    np.random.seed(42)
    n_synth = 1250
    log_ret_synth = np.random.normal(0.0003, 0.012, n_synth)
    vol = np.ones(n_synth) * 0.012
    for t in range(1, n_synth):
        vol[t] = np.sqrt(0.00001 + 0.1 * log_ret_synth[t-1]**2 + 0.85 * vol[t-1]**2)
        log_ret_synth[t] = 0.0003 + vol[t] * np.random.normal()
    prices = 3000 * np.exp(np.cumsum(log_ret_synth))
    log_returns = log_ret_synth[1:]
    dates = pd.date_range('2020-01-02', periods=n_synth, freq='B')

In [ ]:
# Chart: sp500_analysis
# S&P 500 comprehensive analysis (4 panels)
n_ret = len(log_returns)

fig, axes = plt.subplots(2, 2, figsize=(8, 4.0))

axes[0, 0].plot(dates[:len(prices)], prices, color=COLORS['blue'], linewidth=0.8)
axes[0, 0].set_title('S&P 500 Prices', fontsize=9, fontweight='bold')
axes[0, 0].set_ylabel('Price')
axes[0, 0].tick_params(axis='x', rotation=30)

axes[0, 1].plot(dates[1:n_ret+1], log_returns, color=COLORS['red'], linewidth=0.4, alpha=0.8)
axes[0, 1].axhline(0, color=COLORS['gray'], linewidth=0.5, linestyle='--')
axes[0, 1].set_title('Log returns', fontsize=9, fontweight='bold')
axes[0, 1].set_ylabel('Return')
axes[0, 1].tick_params(axis='x', rotation=30)

acf_ret = acf(log_returns, nlags=25)
axes[1, 0].bar(range(len(acf_ret)), acf_ret, color=COLORS['green'], width=0.5, alpha=0.7)
ci = 1.96 / np.sqrt(n_ret)
axes[1, 0].axhline(ci, color=COLORS['red'], linewidth=0.6, linestyle='--')
axes[1, 0].axhline(-ci, color=COLORS['red'], linewidth=0.6, linestyle='--')
axes[1, 0].set_title(r'ACF returns ($\approx 0$)', fontsize=9, fontweight='bold')
axes[1, 0].set_xlabel('Lag')
axes[1, 0].set_ylabel('ACF')

acf_ret2 = acf(log_returns**2, nlags=25)
axes[1, 1].bar(range(len(acf_ret2)), acf_ret2, color=COLORS['orange'], width=0.5, alpha=0.7)
axes[1, 1].axhline(ci, color=COLORS['red'], linewidth=0.6, linestyle='--')
axes[1, 1].axhline(-ci, color=COLORS['red'], linewidth=0.6, linestyle='--')
axes[1, 1].set_title(r'ACF $r_t^2$ (volatility clustering)', fontsize=9, fontweight='bold')
axes[1, 1].set_xlabel('Lag')
axes[1, 1].set_ylabel('ACF')

fig.tight_layout(rect=[0, 0.05, 1, 1])
save_chart(fig, 'sp500_analysis')
plt.show()

In [ ]:
# Chart: returns_distribution
# Returns distribution: histogram vs normal
fig, axes = plt.subplots(1, 2, figsize=(8, 2.8))

axes[0].hist(log_returns, bins=60, color=COLORS['blue'], alpha=0.6, edgecolor='white', density=True, label='S&P 500 returns')
x_range = np.linspace(log_returns.min(), log_returns.max(), 200)
axes[0].plot(x_range, stats.norm.pdf(x_range, log_returns.mean(), log_returns.std()),
             color=COLORS['red'], linewidth=1.5, linestyle='--', label='Normal')
t_params = stats.t.fit(log_returns)
axes[0].plot(x_range, stats.t.pdf(x_range, *t_params),
             color=COLORS['green'], linewidth=1.5, label=f'Student-t (df={t_params[0]:.1f})')
axes[0].set_title('S&P 500 returns distribution', fontsize=9, fontweight='bold')
axes[0].set_xlabel('Return')
axes[0].set_ylabel('Density')
axes[0].legend(fontsize=7, loc='upper right', frameon=False)

stats.probplot(log_returns, dist='norm', plot=axes[1])
axes[1].get_lines()[0].set_color(COLORS['blue'])
axes[1].get_lines()[0].set_markersize(2)
axes[1].get_lines()[1].set_color(COLORS['red'])
axes[1].set_title('QQ-Plot vs Normal', fontsize=9, fontweight='bold')

fig.tight_layout(w_pad=2.0)
save_chart(fig, 'returns_distribution')
plt.show()

In [ ]:
# Chart: volatility_clustering
# Volatility clustering in financial returns
fig, axes = plt.subplots(2, 1, figsize=(7, 3.5))

axes[0].plot(dates[1:len(log_returns)+1], log_returns, color=COLORS['blue'], linewidth=0.4, alpha=0.8)
axes[0].axhline(0, color=COLORS['gray'], linewidth=0.5, linestyle='--')
axes[0].set_title('S&P 500 returns (volatility clustering)', fontsize=9, fontweight='bold')
axes[0].set_ylabel('Return')
axes[0].tick_params(axis='x', rotation=30)

abs_ret = pd.Series(np.abs(log_returns), index=dates[1:len(log_returns)+1])
axes[1].plot(abs_ret.index, abs_ret.values, color=COLORS['orange'], linewidth=0.5, alpha=0.7, label=r'$|r_t|$')
axes[1].plot(abs_ret.rolling(20).mean(), color=COLORS['red'], linewidth=1.2, label='MA(20) $|r_t|$')
axes[1].set_title('Volatility: |returns| and rolling mean', fontsize=9, fontweight='bold')
axes[1].set_xlabel('Date')
axes[1].set_ylabel(r'$|r_t|$')
axes[1].tick_params(axis='x', rotation=30)
axes[1].legend(fontsize=7, loc='upper right', frameon=False)

fig.tight_layout(h_pad=2.0)
save_chart(fig, 'volatility_clustering')
plt.show()